You are asked to implement a visitor called ExpressionPrinter  for printing different mathematical expressions. The range of expressions covers addition and multiplication - please put round brackets around addition operations (but not multiplication ones)! Also, please avoid any blank spaces in output.

Example:

    Input: AdditionExpression(Value(2), Value(3)) 

    Output: (2+3) 

Here is the corresponding unit test:

    class Evaluate(unittest.TestCase):
        def test_simple_addition(self):
            simple = AdditionExpression(Value(2), Value(3))
            ep = ExpressionPrinter()
            ep.visit(simple)
            self.assertEqual("(2+3)", str(ep))

In [7]:
# taken from https://tavianator.com/the-visitor-pattern-in-python/
from abc import ABC


def _qualname(obj):
    """Get the fully-qualified name of an object (including module)."""
    return obj.__module__ + '.' + obj.__qualname__


def _declaring_class(obj):
    """Get the name of the class that declared an object."""
    name = _qualname(obj)
    return name[:name.rfind('.')]


# Stores the actual visitor methods
_methods = {}


# Delegating visitor implementation
def _visitor_impl(self, arg):
    """Actual visitor method implementation."""
    key = (_qualname(type(self)), type(arg))
    if not key in _methods:
        raise Exception('Key % not found' % key)
    method = _methods[key]
    return method(self, arg)


# The actual @visitor decorator
def visitor(arg_type):
    """Decorator that creates a visitor method."""

    def decorator(fn):
        declaring_class = _declaring_class(fn)
        _methods[(declaring_class, arg_type)] = fn

        # Replace all decorated methods with _visitor_impl
        return _visitor_impl

    return decorator

# ↑↑↑ LIBRARY CODE ↑↑↑



class Expression:
    def __init__(self, left, right):
        self.right = right
        self.left = left

    def accept(self, visitor): # Double dispatch visitor implementation
        visitor.visit(self)

class Value(Expression):
    def __init__(self, value):
        self.value = value

class AdditionExpression(Expression):
    def __init__(self, left: Expression, right: Expression):
        self.right = right
        self.left = left


class MultiplicationExpression(Expression):
    def __init__(self, left: Expression, right: Expression):
        self.right = right
        self.left = left


class ExpressionPrinter:
    def __init__(self):
        self.buffer = []

    @visitor(Value)
    def visit(self, expression: Value):
        self.buffer.append(expression.value)

    @visitor(AdditionExpression)
    def visit(self, expression: AdditionExpression):
        self.buffer.append("(")
        expression.left.accept(self) # While this double dispatch is generally required in other programming language it isn't in Python!
        self.buffer.append("+")
        expression.right.accept(self)
        self.buffer.append(")")

    @visitor(MultiplicationExpression)
    def visit(self, expression: Expression):
        expression.left.accept(self)
        self.buffer.append("*")
        expression.right.accept(self)

    def __str__(self):
        return "".join([str(x) for x in self.buffer])

In [8]:
printer = ExpressionPrinter()

expression = AdditionExpression(
    MultiplicationExpression(Value(5), Value(3)),
    AdditionExpression(Value(7), MultiplicationExpression(Value(2), Value(8)))
)

printer.visit(expression)

print(printer)

(5*3+(7+2*8))
